Reading in CSV

In [121]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/iu_pos.csv',encoding='UTF-8',low_memory=False)

df.head(30)

,Unnamed: 0,corpusSeq,docId,unitId,unitStart,unitStartSkipPause,unitStartSkipPauseAlt,unitEnd,unitDur,unitDurSkipPause,...,preClosure,preContinue,preAppeal,curr_POS,prev1_POS,prev2_POS,prev3_POS,init1_POS,init2_POS,init3_POS
0,0,2.0,sbc001,1,0.00,5.294,6.82,9.21,9.21,3.916,...,False,False,False,ADV,NaN,NaN,NaN,SCONJ,NaN,NaN
1,1,3.0,sbc001,1,0.00,5.294,6.82,9.21,9.21,3.916,...,False,False,False,PRON,SCONJ,NaN,NaN,SCONJ,SCONJ,NaN
2,2,4.0,sbc001,1,0.00,5.294,6.82,9.21,9.21,3.916,...,False,False,False,AUX,PRON,SCONJ,NaN,SCONJ,SCONJ,SCONJ
3,3,5.0,sbc001,1,0.00,5.294,6.82,9.21,9.21,3.916,...,False,False,False,PART,AUX,PRON,SCONJ,SCONJ,SCONJ,SCONJ
4,4,6.0,sbc001,1,0.00,5.294,6.82,9.21,9.21,3.916,...,False,False,False,VERB,PART,AUX,PRON,SCONJ,SCONJ,SCONJ
5,5,7.0,sbc001,1,0.00,5.294,6.82,9.21,9.21,3.916,...,False,False,False,PART,VERB,PART,AUX,SCONJ,SCONJ,SCONJ
6,6,8.0,sbc001,1,0.00,5.294,6.82,9.21,9.21,3.916,...,False,False,False,VERB,PART,VERB,PART,SCONJ,SCONJ,SCONJ
7,7,10.0,sbc001,1,0.00,5.294,6.82,9.21,9.21,3.916,...,False,False,False,<UNK>,VERB,PART,VERB,SCONJ,SCONJ,SCONJ
8,8,11.0,sbc001,1,0.00,5.294,6.82,9.21,9.21,3.916,...,False,False,False,<UNK>,<UNK>,VERB,PART,SCONJ,SCONJ,SCONJ
9,9,12.0,sbc001,1,0.00,5.294,6.82,9.21,9.21,3.916,...,False,False,False,ADP,<UNK>,<UNK>,VERB,SCONJ,SCONJ,SCONJ


In [122]:
import re


#removing Non-printable characters
df['text_lower'] = df['text_lower'].str.replace(r'[\x00-\x1F\x7F]', '', regex=True)
df['text_trimmed'] = df['text_trimmed'].str.replace(r'[\x00-\x1F\x7F]', '', regex=True)

words_list = df['text_lower']
dictionary = set(words_list)


print(f'unique words total: {len(dictionary)}')

unique words total: 11435


In [123]:
import numpy as np
from tqdm import tqdm


GLOVE_DATASET_PATH = '/content/drive/MyDrive/IU Prediction/glove.6B.300d.txt'
embeddings_index = {}
word_counter = 0

with open(GLOVE_DATASET_PATH, 'r', encoding='utf-8') as f:
    for line in tqdm(f):
        try:
            values = line.split()
            word = values[0]
            # Check if the word is in the dictionary before processing
            if word in dictionary:
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs
            word_counter += 1
        except ValueError:
            continue

print(f'Found {len(embeddings_index)} word vectors matching the specified dictionary.')
missing_embeddings = [word for word in dictionary if word not in embeddings_index]

400000it [00:12, 32267.45it/s]

Found 10761 word vectors matching the specified dictionary.


In [97]:
file_path = '/content/missing_embeddings.txt'

# Open the file in write mode ('w') and write each word on a new line
with open(file_path, 'w') as file:
    for word in missing_embeddings:
        file.write(f"{word}\n")

print(f"File saved as {file_path}")

File saved as /content/missing_embeddings.txt


In [130]:
words_in_lower_not_trimmed = set(df['text_lower']) - set(df['text_trimmed'])


average_embedding_index = {word: embeddings_index[word] for word in words_in_lower_not_trimmed if word in embeddings_index}


embedding_size = len(next(iter(embeddings_index.values())))
sum_of_embeddings = np.zeros(embedding_size)

for embedding in average_embedding_index.values():
    sum_of_embeddings += embedding

average_embedding = sum_of_embeddings /len(average_embedding_index)

print("Average embedding:", average_embedding)

Average embedding: [-3.30244701e-02  1.42202539e-02 -2.29614771e-02 -1.50466088e-02
 -2.01374847e-02  2.69815228e-02  7.94247969e-03 -1.83650753e-02
  1.17353015e-02 -4.59316282e-01  1.60861505e-03 -1.58418749e-03
 -1.11033090e-02 -4.82848267e-03  1.12327150e-03 -1.31871458e-03
 -2.91951217e-02 -2.52593745e-02  1.75911059e-02  2.91704853e-02
  1.72525052e-02  8.60677419e-02 -5.05053865e-03  3.25122330e-02
 -3.78922513e-02 -2.30845965e-03  1.34587870e-02 -1.81168659e-02
  1.28255605e-02  2.24885458e-02 -2.17757235e-02  6.70640664e-02
 -6.19762205e-02 -3.66856257e-02 -2.10470383e-01  8.40925648e-02
 -5.37338698e-02 -8.34402930e-03 -5.41177016e-03  2.22924956e-02
  1.77969731e-02 -2.29148476e-02 -2.94084383e-02 -5.52557758e-02
  1.74111417e-02  4.99174846e-02  8.39310048e-02  2.84817904e-02
 -2.68212194e-02  1.21568547e-02  3.83479722e-02 -3.88797848e-02
  6.05353212e-02 -2.78847291e-02 -5.26407219e-02  2.08228202e-02
 -1.26449432e-03  1.06875998e-02  2.66701324e-02  5.34585595e-02
  5.85

In [131]:
for word in missing_embeddings:
    embeddings_index[word] = average_embedding

In [132]:
missing_embeddings = [word for word in dictionary if word not in embeddings_index]

missing_embeddings

[]

In [133]:
unembedded_words_in_text_trimmed = [word for word in df['text_trimmed'] if word not in embeddings_index]


print(set(unembedded_words_in_text_trimmed))

{'<UNK>'}


In [134]:
UNK_embedding = embeddings_index.get('<UNK>', None)
print(UNK_embedding)

In [137]:
embeddings_index['<UNK>'] = average_embedding

In [ ]:
UNK_embedding = embeddings_index.get('<UNK>', None)
print(UNK_embedding)

In [145]:
words_to_keep = set(df['text_trimmed'])
print(len(words_to_keep))

final_embeddings_index = {word: embeddings_index[word] for word in words_to_keep if word in embeddings_index}

200
